In [561]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
pd.set_option('display.max_columns', 500)

import datetime

from __future__ import division

import seaborn as sns

In [587]:
def volume_time_converter(df, column='time', shift=False):
    
    # Initialize lists to store splitted information
    date_list = []
    month_list = []
    day_list = []
    hour_list = []
    minute_list = []
    rounded_min_list = []
    rounded_hour_list = []
    
    def split_date(date):
        
        parts = date.split(" ")
        day_part = parts[0]
        clock_part = parts[1]
        
        day_parts = day_part.split("-")
        month_list.append(day_parts[1])
        day_list.append(day_parts[2])
        
        clock_parts = clock_part.split(':')
        hour_list.append(clock_parts[0])
        minute_list.append(clock_parts[1])
        rounded_hour_list.append(int(clock_parts[0]) // 3 * 3)
        rounded_min_list.append(int(clock_parts[1]) // 20 * 20)
        
        date_list.append(day_part)
    
    # split 'time' feature and store into list
    for date in df[column]:
        split_date(date)
        
    # Add arrays into the 'volume' df
    df['month'] = np.array(month_list)
    df['day'] = np.array(day_list)
    df['hour'] = np.array(hour_list)
    df['minute'] = np.array(minute_list)
    df['date'] = np.array(date_list)
    df['rounded_hour'] = np.array(rounded_hour_list)
    df['rounded_min'] = np.array(rounded_min_list)
    
    # Add an colume which combine 'date' and 'rounded_hour'
    slash_list = np.array(['-'] * len(df['date']))
    df['date_and_rounded_hour'] = df['date'].astype(str) + slash_list + df['rounded_hour'].astype(str)
    
    df['window_time'] = df['date'].astype(str) + slash_list + df['hour'].astype(str) + slash_list + \
                            df['rounded_min'].astype(str)
        
    df['window_time_formatted'] = df['window_time'].apply(
            lambda t : datetime.datetime.strptime(t, '%Y-%m-%d-%H-%M'))
    
    # Shift time by 7 days(for submission sample data only)
    if shift:
        df['window_time_formatted'] = df['window_time_formatted'].apply(
            lambda t : (t + datetime.timedelta(days=7)))
        df['date'] = df['window_time_formatted'].apply(lambda t : t.date().strftime('%Y-%m-%d'))
        df['date_and_rounded_hour'] = df['date'].astype(str) + slash_list + df['rounded_hour'].astype(str)
        
    
    return df

In [588]:
def weather_time_converter(df):
    
    slash_list = np.array(['-'] * len(df))
    df['date_and_rounded_hour'] = df['date'] + slash_list + df['hour'].astype(str)
    
    df['rounded_min'] = np.array([0] * len(df))
    
    df['window_time'] = df['date'].astype(str) + slash_list + df['hour'].astype(str) + slash_list + \
                            df['rounded_min'].astype(str)
    
    return df

In [589]:
def traj_time_converter(df):
    
    # Initialize 'direction' as 0
    df['direction'] = np.array([0] * len(df))
    
    # Drop milesecond and reformat second
    df['travel_time'] = (df['travel_time'] // 1 * 1).astype(int)
    
    # Reformat 'starting_time'
    df['starting_time_formatted'] = df['starting_time'].apply(
        lambda t : datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S'))
    
    # Reformat 'travel_time'
    df['travel_time_formatted'] = df['travel_time'].apply(lambda s : datetime.timedelta(seconds=s))
    
    # Add 'end_time' 
    df['end_time_formatted'] = (df['starting_time_formatted'] + df['travel_time_formatted'])

    # Create time features
    year_list = df['end_time_formatted'].apply(lambda t : t.year).astype(str)
    month_list = df['end_time_formatted'].apply(lambda t : t.month).astype(str)
    day_list = df['end_time_formatted'].apply(lambda t : t.day).astype(str)
    hour_list = df['end_time_formatted'].apply(lambda t : t.hour).astype(str)
    min_list = df['end_time_formatted'].apply(lambda t : t.minute).astype(str)
    rounded_hour_list = hour_list.astype(int).apply(lambda t : t // 3 * 3).astype(str)
    rounded_min_list = min_list.astype(int).apply(lambda t : t // 20 * 20).astype(str)
    slash_list = np.array(['-'] * len(df))
    
    df['year'] = year_list
    df['month'] = month_list
    df['day'] = day_list
    df['hour'] = hour_list
    df['min'] = min_list
    df['rounded_hour'] = rounded_hour_list
    df['rounded_min'] = rounded_min_list
    

    # Create 'window_time'
    df['window_time'] = year_list + slash_list + month_list + slash_list \
            + day_list + slash_list + hour_list + slash_list + rounded_min_list
        
    df['window_time_formatted'] = df['window_time'].apply(
            lambda t : datetime.datetime.strptime(t, '%Y-%m-%d-%H-%M'))
    
    df['date_and_rounded_hour'] = year_list + slash_list + month_list + slash_list \
            + day_list + slash_list + rounded_hour_list
    
    # Shift window_time by 2 hours here(as only last 2 hours data are provided in test set)
    df['window_time_formatted'] = df['window_time_formatted'].apply(
            lambda t : (t + datetime.timedelta(hours=2)))

    return df

## Merge and group data

In [592]:
# Merge volume and weather 
def group_vol_wea(volume, weather):
    
    df = pd.merge(volume, weather, on='date_and_rounded_hour', suffixes=('', '_y'), how='left')
    
    df = df.groupby(['window_time_formatted', 'tollgate_id', 'direction'])
    
    df_gp = df.agg('mean').join(pd.DataFrame(df.size(), columns=['count']))
    
    # Put index as column
    df_gp['direction'] = df_gp.index.get_level_values('direction')
    df_gp['tollgate_id'] = df_gp.index.get_level_values('tollgate_id')
    df_gp['window_time_formatted'] = df_gp.index.get_level_values('window_time_formatted')
    
    return df_gp

In [594]:
def group_traj(df):
    
    df = df.groupby(['window_time_formatted', 'tollgate_id', 'direction'])
    df_gp = df.agg('mean').join(pd.DataFrame(df.size(), columns=['count']))
    
    # Put index as column
    df_gp['direction'] = df_gp.index.get_level_values('direction')
    df_gp['tollgate_id'] = df_gp.index.get_level_values('tollgate_id')
    df_gp['window_time_formatted'] = df_gp.index.get_level_values('window_time_formatted')
    
    return df_gp

In [596]:
# Merge volume_weather and trajectories
def merge_vwt(vw_gp, t_gp):
    
    vwt = pd.merge(vw_gp, t_gp, on=['window_time_formatted', 'tollgate_id', 'direction'], suffixes=('', '_y'), how='left')
    
    # Drop useless columns
    drop_columns = ['vehicle_id', 'vehicle_model', 'vehicle_type', 'hour_y', 'rounded_min_y', 'has_etc']
    for col in drop_columns:
        if col in vwt.columns:
            vwt.drop([col], axis=1, inplace=True)
            
    vwt.rename(columns={'count_y':'traj_count'}, inplace=True)
    
    return vwt

## Create new features

In [598]:
# Add features: 'weekday', 'is_festival', 'is_working_day'
def add_features(vwt):
    
    vwt['month'] = vwt['window_time_formatted'].apply(lambda t : t.month)
    vwt['day'] = vwt['window_time_formatted'].apply(lambda t : t.day)
    
    # Create 'weekday' column in DateFrame(0 stands for Sunday; 1 stands for Monday and 2 stands for Tuesday, etc...)
    if (9 in list(vwt['month'].unique())):
        sept = vwt[vwt['month'] == 9]
        weekday1 = ((sept['day'] + 3) % 7).values
        octo = vwt[vwt['month'] == 10]
        weekday2 = ((octo['day'] + 5) % 7).values
        vwt['weekday'] = np.append(weekday1, weekday2)
    else:
        weekday2 = ((vwt['day'] + 5) % 7).values
        vwt['weekday'] = np.array(weekday2)
        
    # Create 'is_festival' column
    vwt['is_festival'] = np.array([0] * len(vwt))
    sep_days = [15, 16, 17, 30]
    oct_days = [1, 2, 3, 4, 5, 6, 7, 8]
    if (9 in list(vwt['month'].unique())):
        for day in sep_days:
            vwt.loc[((vwt['month'] == 9) & (vwt['day'] == day)), 'is_festival'] = 1
    for day in oct_days:
        vwt.loc[((vwt['month'] == 10) & (vwt['day'] == day)), 'is_festival'] = 1
        
    # Construct 'is_working_day' column
    vwt['is_working_day'] = np.array([0] * len(vwt))
    vwt.loc[((vwt['weekday'] < 5) & (vwt['weekday'] > 0)), 'is_working_day'] = 1
    vwt.loc[vwt['is_festival'] == 1, 'is_working_day'] = 0
    vwt.loc[((vwt['month'] == 9) & (vwt['day'] == 18)), 'is_working_day'] = 1
    vwt.loc[((vwt['month'] == 10)
                        & ((vwt['day'] == 8) | (vwt['day'] == 9))), 'is_working_day'] = 1
    
    return vwt

## Fill NA

#### more needs to be done. 'travel_time' nas are not supposed to filled with meadian

In [606]:
def fill_na(vwt):
    
#     weather_features = ['pressure', 'sea_pressure', 'wind_direction', 'wind_speed', 'temperature', 'rel_humidity', 'precipitation']
#     for fea in weather_features:
#         vwt[fea] = vwt.groupby("rounded_hour").transform(lambda x: x.fillna(x.mean()))
        
    vwt = vwt.fillna(vwt[:vwt.shape[0]].mean())
    
#     vwt_out = vwt[vwt['direction'] == 1]
#     vwt_in = vwt[vwt['direction'] == 0]
    
#     vwt_out.drop(['travel_time', 'traj_count'], axis=1, inplace=True)

    return vwt

## Create dummy

In [608]:
def create_dummy(vwt):
    
    vwt['hour'] = vwt['window_time_formatted'].apply(lambda t : t.hour)
    
    # Split numerical data
    vwt['wind_direction'] = vwt['wind_direction'] // 15 * 15
    vwt['precipitation'] = vwt['precipitation'] // 1 * 1
    vwt['wind_speed'] = vwt['wind_speed'] // 1 * 1
    vwt['temperature'] = vwt['temperature'] // 2 * 2
    vwt['rel_humidity'] = vwt['rel_humidity'] // 10 * 10
    
    day_list = vwt['day']
    hour_list = vwt['hour']
    weekday_list = vwt['weekday']
    
    # Create dummies
    vwt = pd.get_dummies(data=vwt, columns=['weekday', 'hour', 'rounded_min', 'wind_direction',
                                                        'wind_speed', 'temperature', 'rel_humidity', 'precipitation'])
    vwt['day'] = day_list
    vwt['hour'] = hour_list
    vwt['weekday'] = weekday_list
    
    return vwt

## Create feature 'history average'

In [611]:
def create_his_ave(df):

    df = df[df['is_festival'] == 0]
    
    df['time'] = df['window_time_formatted'].apply(lambda t : t.time())
    df_his = df.groupby(['time', 'tollgate_id', 'direction', 'weekday']).agg('mean')
    df_his.rename(columns={'count':'his_ave'}, inplace=True)
    
    df_his['direction'] = df_his.index.get_level_values('direction')
    df_his['tollgate_id'] = df_his.index.get_level_values('tollgate_id')
    df_his['time'] = df_his.index.get_level_values('time')
    df_his['weekday'] = df_his.index.get_level_values('weekday')

    
    return df_his[['time', 'tollgate_id', 'direction', 'his_ave', 'weekday']]

## Add feature 'his_ave'

In [613]:
def add_his_ave(df, vwt_his):
    
    df['time'] = df['window_time_formatted'].apply(lambda t : t.time())
    df = pd.merge(df, vwt_his, on=['time', 'tollgate_id', 'direction', 'weekday'], suffixes=('', '_y'), how='left' )
    
    return df

# Prepare training data

In [646]:
links = pd.read_csv('training/links (table 3).csv')
routes = pd.read_csv('training/routes (table 4).csv')
trajectories = pd.read_csv('training/trajectories(table 5)_training.csv')
volume = pd.read_csv('training/volume(table 6)_training.csv')
weather = pd.read_csv('training/weather (table 7)_training.csv')

volume_late = pd.read_csv('dataSet_phase2/volume(table 6)_training2.csv')
trajectories_late = pd.read_csv('dataSet_phase2/trajectories(table_5)_training2.csv')
weather_late = pd.read_csv('dataSet_phase2/weather (table 7)_2.csv')

volume_late.rename(columns={'tollgate':'tollgate_id', 'date_time':'time',
                            'is_etc':'has_etc', 'model':'vehicle_model'}, inplace=True)

In [647]:
volume = pd.concat([volume, volume_late])
weather = pd.concat([weather, weather_late])
trajectories = pd.concat([trajectories, trajectories_late])

# Convert 'wind_direction' outlier to 360
weather.loc[weather['wind_direction'] > 360, 'wind_direction'] = 360

In [651]:
v = volume.copy()
t = trajectories.copy()
w = weather.copy()

v = volume_time_converter(v)
t = traj_time_converter(t)
w = weather_time_converter(w)

vw_gp = group_vol_wea(v, w)
t_gp = group_traj(t)
vwt = merge_vwt(vw_gp, t_gp)

vwt = add_features(vwt)
vwt = fill_na(vwt)
vwt = create_dummy(vwt)
vwt_his = create_his_ave(vwt)
vwt = add_his_ave(vwt, vwt_his)
vwt = vwt[vwt['hour'].isin([8, 9, 17, 18])]

C:\Users\THINKPAD\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Prepare submission data

In [652]:
sub_volume = pd.read_csv('submission_sample_volume.csv')

sub_weather = pd.read_csv('dataSet_phase2/weather (table 7)_2.csv')

sub_trajectories = pd.read_csv('dataSet_phase2/trajectories(table 5)_test2.csv')

In [655]:
sub_volume['time'] = sub_volume['time_window'].apply(
            lambda t : t.split(',')[0].split('[')[1])

In [656]:
sub_v = sub_volume.copy()
sub_w = sub_weather.copy()
sub_t = sub_trajectories.copy()

In [657]:
sub_v = volume_time_converter(sub_v, column='time', shift=True)
sub_v.rename(columns={'tollgate':'tollgate_id'}, inplace=True)
sub_t = traj_time_converter(sub_t)
sub_w = weather_time_converter(sub_w)
sub_vw_gp = group_vol_wea(sub_v, sub_w)
sub_t_gp = group_traj(sub_t)
sub_vwt = merge_vwt(sub_vw_gp, sub_t_gp)
sub_vwt = add_features(sub_vwt)
sub_vwt = fill_na(sub_vwt)
sub_vwt = create_dummy(sub_vwt)
sub_vwt = add_his_ave(sub_vwt, vwt_his)
sub_vwt = sub_vwt[sub_vwt['hour'].isin([8, 9, 17, 18])]

# Save data

In [ ]:
# vwt.to_csv('train_vwt.csv')
# sub_vwt.to_csv('sub_vwt.csv')

In [628]:
df = vwt.copy()
vwt_copy = vwt.copy()

In [629]:
features = ['traj_count', 'travel_time']

In [630]:
na_mask = df[features[0]].isnull()

In [631]:
df = df[~na_mask][['time', 'tollgate_id', 'direction', features[0]]]

In [632]:
df = df.groupby(['time', 'tollgate_id', 'direction']).agg('mean')

In [633]:
df['time'] = df.index.get_level_values('time')
df['tollgate_id'] = df.index.get_level_values('tollgate_id')
df['direction'] = df.index.get_level_values('direction')

In [634]:
vwt_copy = pd.merge(vwt_copy, df, on=['time', 'tollgate_id', 'direction', features[0]], suffixes=('', '_y'), how='left')

In [649]:
# vwt_copy[vwt_copy['direction'] == 0].isnull().sum()

In [650]:
# sub_vwtc[sub_vwtc['direction'] == 0].isnull().sum()

In [660]:
vv = vwt.copy()

In [662]:
vv['date'] = vwt['window_time_formatted'].apply(lambda t : t.date())